In [1]:
import jax
import jax.numpy as jnp
import inference.mlp as mlp

In [2]:
c = 1
h = 60
w = 80
sizes = [512, 256, 128]

key = jax.random.PRNGKey(42)

weights = mlp.qs_mlp(c, h, w, sizes, key, 1)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!
2024-05-18 21:25:29.685595: W pjrt_plugin/src/mps_client.cc:563] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!


Metal device set to: Apple M2

systemMemory: 24.00 GB
maxCacheSize: 8.00 GB



In [6]:
import pickle

# Specify the file path
file_path = "weights/weights.pkl"

# Save the weights to the file
with open(file_path, "wb") as file:
    pickle.dump(weights, file)

# Load the weights from the file
with open(file_path, "rb") as file:
    loaded_weights = pickle.load(file)

# Check if the loaded weights are the same as the original weights
for i in range(len(weights)):
    for j in range(len(weights[i])):
        assert jnp.allclose(weights[i][j], loaded_weights[i][j])

In [ ]:
#tell me how big in bytes that file is in bytes
import os
print(os.path.getsize('weights/weights2.bin'))


20320296


In [ ]:
jnp.ones((2, c, h, w), dtype=jnp.float32).flatten().shape


NameError: name 'jnp' is not defined